# Analyse de données de films

Nous proposons de travailler sur des données décrivant des films. Les possibilités sont larges et vous êtes évalués sur vos propositions et votre méthodologie plus que sur vos résultats.

Les données de départ sont disponibles sur:
https://grouplens.org/datasets/movielens/
au format CSV. 

Nous nous intéresserons en particulier au jeu de données: **MovieLens 20M Dataset**. Dans ce jeu de données, vous disposez entre autre de:
* Idendifiant du film dans IMdb et TMdb (ça sera important ensuite)
* Catégorie(s) du film
* Titre du film
* Notes données par les internautes aux films

Afin de rendre le projet plus intéressant, nous ajoutons des données sur les acteurs et producteurs associés aux films (récupéré sur TMdb). Ces données sont disponibles sur les liens suivants:

http://webia.lip6.fr/~guigue/film_v2.pkl <br>
http://webia.lip6.fr/~guigue/act_v2.pkl <br>
http://webia.lip6.fr/~guigue/crew_v2.pkl

Ces fichiers contiennent respectivement : une nouvelle description des films (dont l'identifiant TMdb et la note moyenne donnée par les internautes, la date de sortie,...), une description des acteurs de chaque film et une description des équipes (scénariste, producteur, metteur en scène) pour chaque film.

Ces données sont des listes de taille 26908, chaque élément de la liste correspondant à un dictionnaire dont vous étudierez les clés pour récupérer les informations utiles.

**ATTENTION** Les contraintes de récupération d'informations en ligne font que la base MovieLens compte 27278 films mais les fichiers ci-dessus n'en comptent que 26908. Le plus simple est probablement d'éliminer les films de MovieLens qui ne sont pas dans cette seconde base.

## Consignes générales pour l'analyse des données

Vous devez proposer plusieurs analyses des données, qui devront à minima utiliser les
 techniques suivantes:
 
1. Mettre en forme les données pour identifier les acteurs et les catégories, les indexer
1. Traiter au moins un problème de régression supervisé (par exemple la prédiction de la note moyenne donnée à un film par les internautes).
1. Traiter au moins un problème de classification supervisé (par exemple la prédiction de la catégorie d'un film)
1. Utiliser les données catégorielles (catégories, acteurs,...) de manière discrète ET de manière coninue (*dummy coding*) dans des approches différentes
1. Proposer au moins une approche de catégorisation non supervisée (pour regrouper les acteurs par exemple)
1. Mener une campagne d'expérience permettant de comparer les performances sur un problème en fonction des valeurs d'un paramètre (et donc, in fine, trouver la meilleure valeur du paramètre)
1. Proposer quelques illustrations

In [133]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils
%matplotlib inline
import pickle as pkl


## Chargement des données (base MovieLens + enrichissements)

In [134]:
# chargement des données du genome_scores
#fname_genome_scores ="C:/Users/ING-FLUX/Desktop/IA/projet/ml-20m/genome-scores.csv"
#genome_scores = pd.read_csv(fname_genome_scores, encoding='utf8')

In [135]:
#chargement des données du genome_tags
#fname_genome_tags ="C:/Users/ING-FLUX/Desktop/IA/projet/ml-20m/genome-tags.csv"
#genome_tags = pd.read_csv(fname_genome_tags, encoding='utf8')

In [136]:
#chargement des données de links
fname_links ="C:/Users/ING-FLUX/Desktop/IA/projet/ml-20m/links.csv"
links = pd.read_csv(fname_links, encoding='utf8')

In [137]:
#chargement des données de movies
fname_movies ="C:/Users/ING-FLUX/Desktop/IA/projet/ml-20m/movies.csv"
movies = pd.read_csv(fname_movies, encoding='utf8')

In [138]:
#chargement des données de ratings
fname_ratings ="C:/Users/ING-FLUX/Desktop/IA/projet/ml-20m/ratings.csv"
ratings = pd.read_csv(fname_ratings, encoding='utf8')

In [74]:
#chargement des données de tags
#fname_tags ="C:/Users/ING-FLUX/Desktop/IA/projet/ml-20m/tags.csv"
#tags = pd.read_csv(fname_tags, encoding='utf8')

In [139]:
# Chargement des données complémentaires des acteurs
fname_acteurs = "act_v2.pkl"
acteurs = pkl.load(open(fname_acteurs, "rb"))

In [140]:
# Chargement des données complémentaires des crew
fname_crew = "crew_v2.pkl"
crew = pkl.load(open(fname_crew, "rb"))

In [145]:
# Chargement des données complémentaires des acteurs
fname_film = "film_v2.pkl"
film= pkl.load(open(fname_film, "rb"))

In [142]:
dicFilmActeur=dict()# dictionnaire contenant pour chaque indice de film les acteurs y jouant dedans
ind=0
res=[]
for film in acteurs:
    if(film!=[]):
        dicFilmActeur[ind]=film[0]['name']
    else:
        dicFilmActeur[ind]=""
    ind+=1




In [143]:
dicRealisateurFilm=dict()# dictionnaire contenant pour chaque indice de film les acteurs y jouant dedans
ind=0
for i in crew :
    if(i!=[]):
        x=i[0]
        if(x['job']=="Director"):
            dicRealisateurFilm[ind]=x['name']
        else:
            dicRealisateurFilm[ind]=""
    else:
        dicRealisateurFilm[ind]=""
    ind+=1


In [146]:
for i in range(len(film)):
    tmp={}
    tmp['acteur']=dicFilmActeur[i]
    tmp['realisateur']=dicRealisateurFilm[i]
    
    if film[i]['original_title'] != film[i]['title']:
        tmp['title_modified']=1
    else:
        tmp['title_modified']=-1
    film[i].update(tmp)

In [147]:
df_film=pd.DataFrame(film)

In [148]:
genresF = ["Action", "Adventure", "Animation", "Children", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War","Western"]
for g in genresF:
    movies[g] = movies.apply(lambda r: 2*int(g in r["genres"]) - 1, axis=1)
movies.drop("genres", axis=1, inplace=True)

In [149]:
movies["Average rating"] = ratings.groupby("movieId")["rating"].mean()
movies["Number of ratings"] = ratings.groupby("movieId")["rating"].size()

movies["Average rating"].fillna(0, inplace = True)
movies["Number of ratings"].fillna(0, inplace = True)

In [150]:
df_film.drop(columns=['title'])


,acteur,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,realisateur,release_date,title_modified,video,vote_average,vote_count
0,Tom Hanks,False,/dji4Fm0gCDVb9DQQMRvAI8YNnTz.jpg,"[16, 35, 10751]",862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",22.773,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,John Lasseter,1995-10-30,-1,False,7.9,9550
1,Robin Williams,False,/7k4zEgUZbzMHawDaMc9yIkmY1qR.jpg,"[12, 14, 10751]",8844,en,Jumanji,When siblings Judy and Peter discover an encha...,2.947,/vgpXmVaVyUL7GGiDeiK1mKEKzcX.jpg,Joe Johnston,1995-12-15,-1,False,7.1,5594
2,Walter Matthau,False,/1ENbkuIYK2taNGGKNMs2hw6SaJb.jpg,"[35, 10749]",15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,6.076,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,Howard Deutch,1995-12-22,-1,False,6.5,140
3,Whitney Houston,False,/u0hQzp4xfag3ZhsKKBBdgyIVvCl.jpg,"[35, 18, 10749]",31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",2.917,/4wjGMwPsdlvi025ZqR4rXnFDvBz.jpg,Forest Whitaker,1995-12-22,-1,False,6.1,55
4,Steve Martin,False,/cZs50rEk4T13qWedon0uCnbYQzW.jpg,[35],11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,6.817,/e64sOI48hQXyru7naBFyssKFxVd.jpg,,1995-02-10,-1,False,6.1,288
5,Al Pacino,False,/jMzVSwQp1lLVq9fnQQ4yOjr1YZ2.jpg,"[28, 80, 18, 53]",949,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",13.666,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,Michael Mann,1995-12-15,-1,False,7.8,3002
6,Harrison Ford,False,/hSy5yZG18ogNQn1tHSlxSqV24cf.jpg,"[35, 10749]",11860,en,Sabrina,An ugly duckling having undergone a remarkable...,6.177,/jQh15y5YB7bWz1NtffNZmRw0s9D.jpg,Sydney Pollack,1995-12-15,-1,False,6.1,260
7,Jonathan Taylor Thomas,False,/43r8WYBhOrj0SLSTuShynuWj6Z.jpg,"[28, 12, 18, 10751]",45325,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",3.567,/sGO5Qa55p7wTu7FJcX4H4xIVKvS.jpg,Peter Hewitt,1995-12-22,-1,False,5.3,73
8,Jean-Claude Van Damme,False,/y6A4PUAD61r15CgtuuQhWxLh6Vx.jpg,"[28, 12, 18, 53]",9091,en,Sudden Death,When a man's daughter is suddenly taken during...,5.890,/ridz4IucWay8dBP5t68rGYykCvi.jpg,Peter Hyams,1995-10-27,-1,False,5.7,279
9,Pierce Brosnan,False,/dA9I0Vd9OZzRQ2GyGcsFXdKGMz3.jpg,"[12, 28, 53]",710,en,GoldenEye,James Bond must unmask the mysterious head of ...,16.629,/5c0ovjT41KnYIHYuF4AWsTe3sKh.jpg,Martin Campbell,1995-11-16,-1,False,6.8,1853


In [151]:
dataFrame=movies
dataFrame


,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Average rating,Number of ratings
0,1,Toy Story (1995),-1,1,1,1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,0.000000,0.0
1,2,Jumanji (1995),-1,1,-1,1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,3.921240,49695.0
2,3,Grumpier Old Men (1995),-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,3.211977,22243.0
3,4,Waiting to Exhale (1995),-1,-1,-1,-1,1,-1,-1,1,...,-1,-1,-1,1,-1,-1,-1,-1,3.151040,12735.0
4,5,Father of the Bride Part II (1995),-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,2.861393,2756.0
5,6,Heat (1995),1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,1,-1,-1,3.064592,12161.0
6,7,Sabrina (1995),-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,3.834930,23899.0
7,8,Tom and Huck (1995),-1,1,-1,1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,3.366484,12961.0
8,9,Sudden Death (1995),1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,3.142049,1415.0
9,10,GoldenEye (1995),1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,1,-1,-1,3.004924,3960.0


In [152]:
dataFrame=pd.merge(dataFrame,links,left_on='movieId',right_on='movieId')
dataFrame=pd.merge(dataFrame,df_film,left_on='tmdbId',right_on='id')

In [153]:
dataFrame.columns

Index(['movieId', 'title_x', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western', 'Average rating', 'Number of ratings', 'imdbId', 'tmdbId',
       'acteur', 'adult', 'backdrop_path', 'genre_ids', 'id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'realisateur', 'release_date', 'title_y',
       'title_modified', 'video', 'vote_average', 'vote_count'],
      dtype='object')

In [154]:
dataFrame=dataFrame.drop(columns=['adult','backdrop_path', 'genre_ids','imdbId', 'tmdbId','title_y', 'video', 'overview','id','poster_path'])

In [155]:
dataFrame.to_csv('BaseComplete.csv')

In [156]:
dataFrame[["realisateur",'movieId']].head()

,realisateur,movieId
0,John Lasseter,1
1,Joe Johnston,2
2,Howard Deutch,3
3,Forest Whitaker,4
4,,5


In [157]:
df_FilmActeur=dataFrame[['movieId',"acteur"]]

In [158]:
df_FilmRealisateur=dataFrame[['movieId',"realisateur"]]

In [159]:
liste=df_FilmRealisateur['realisateur'].values

In [160]:
df_FilmActeur

,movieId,acteur
0,1,Tom Hanks
1,2,Robin Williams
2,3,Walter Matthau
3,4,Whitney Houston
4,5,Steve Martin
5,6,Al Pacino
6,7,Harrison Ford
7,8,Jonathan Taylor Thomas
8,9,Jean-Claude Van Damme
9,10,Pierce Brosnan
